# Prometheus Relation Model

`sbt publishM2`

Repeat if the project has been updated, and also do:

`rm -rf ~/.m2/repository/sonymobile`

`rm -rf metastore_db`

(*Important*: `kernel -> restart all` otherwise the previously used version of the prometheus-relation-model lib will be used again! Very confusing!) <- not sure if this is correct

`prometheus-relation-model` requires [docforia](https://github.com/marcusklang/docforia), which must be git-cloned and `mvn install`:ed first (not yet on Maven repository unfortunately).

In [1]:
%AddDeps sonymobile prometheus-relation-model_2.10 0.0.1-SNAPSHOT --transitive --repository file:/Users/axel/.m2/repository

Marking sonymobile:prometheus-relation-model_2.10:0.0.1-SNAPSHOT for download
Preparing to fetch from:
-> file:/var/folders/28/5mj8jbrd13z_nssxk35nd25c0000gn/T/toree_add_deps8523781516040407919/
-> file:/Users/axel/.m2/repository
-> https://repo1.maven.org/maven2
-> New file at /Users/axel/.m2/repository/se/lth/cs/nlp/docforia/1.0-SNAPSHOT/docforia-1.0-SNAPSHOT.jar
-> New file at /Users/axel/.m2/repository/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
-> New file at /Users/axel/.m2/repository/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.jar
-> New file at /Users/axel/.m2/repository/junit/junit/4.11/junit-4.11.jar
-> New file at /Users/axel/.m2/repository/org/xerial/snappy/snappy-java/1.1.2.1/snappy-java-1.1.2.1.jar
-> New file at /Users/axel/.m2/repository/sonymobile/prometheus-relation-model_2.10/0.0.1-SNAPSHOT/prometheus-relation-model_2.10-0.0.1-SNAPSHOT.jar
-> New file at /Users/axel/.m2/repository/it/unimi/dsi/fastutil/6.3/fastutil-6.3.jar
-> New file at /var

In [5]:
%AddDeps se.lth.cs.nlp docforia 1.0-SNAPSHOT --transitive --repository file:/Users/axel/.m2/repository

Marking se.lth.cs.nlp:docforia:1.0-SNAPSHOT for download
Preparing to fetch from:
-> file:/var/folders/28/5mj8jbrd13z_nssxk35nd25c0000gn/T/toree_add_deps3852866345726084937/
-> file:/Users/axel/.m2/repository
-> https://repo1.maven.org/maven2
-> New file at /Users/axel/.m2/repository/se/lth/cs/nlp/docforia/1.0-SNAPSHOT/docforia-1.0-SNAPSHOT.jar
-> New file at /Users/axel/.m2/repository/com/fasterxml/jackson/core/jackson-annotations/2.7.0/jackson-annotations-2.7.0.jar
-> New file at /Users/axel/.m2/repository/junit/junit/4.11/junit-4.11.jar
-> New file at /Users/axel/.m2/repository/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.jar
-> New file at /Users/axel/.m2/repository/com/fasterxml/jackson/core/jackson-databind/2.7.3/jackson-databind-2.7.3.jar
-> New file at /Users/axel/.m2/repository/com/fasterxml/jackson/core/jackson-core/2.7.3/jackson-core-2.7.3.jar
-> New file at /Users/axel/.m2/repository/com/google/protobuf/protobuf-java/2.6.1/protobuf-java-2.6.1.jar
-> New file at /Users/a

In [15]:
import com.sony.relationmodel.CorpusReader

val docs = CorpusReader.readCorpus("/Users/axel/utveckling/prometheus/data/wikipedia-corpus-herd/sv")(sqlContext, sc)

In [16]:
val gp = docs.filter(_.uri() == "urn:wikidata:Q53747").collect()

In [17]:
println(gp)

[Lse.lth.cs.docforia.Document;@4db7f69a


In [5]:
import org.apache.spark.mllib.linalg.{SparseVector, Vector, Vectors}
val sparseV = Vectors.sparse(50, Array(0, 2), Array(1.0, 3.0))
println(sparseV.getClass)

class org.apache.spark.mllib.linalg.SparseVector


# Feature Extraction

In [18]:
import com.sony.prometheus.stages.TrainingDataExtractor
val relations = RelationsReader.readRelations("/Users/axel/utveckling/prometheus/data/0.4.0/entities")
val sentences = TrainingDataExtractor.extract(docs, relations)


Name: Compile Error
Message: <console>:34: error: object prometheus is not a member of package com.sony
         import com.sony.prometheus.stages.TrainingDataExtractor
                         ^
StackTrace: 

# One-vs-All

In [6]:
import java.util.concurrent.TimeUnit.{NANOSECONDS => NANO}

import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.ml.classification.{OneVsRest, LogisticRegression}
import org.apache.spark.ml.util.MetadataUtils
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.sql.DataFrame

In [7]:
val inputData = sqlContext.read.format("libsvm").load("iris.scale")
val data = inputData.randomSplit(Array(0.8, 0.2))

val Array(train, test) = data.map(_.cache())

Name: org.apache.hadoop.mapred.InvalidInputException
Message: Input path does not exist: file:/Users/axel/utveckling/prometheus/relation-model/iris.scale
StackTrace: org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:285)
org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:228)
org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:313)
org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:202)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
scala.Option.getOrElse(Option.scala:120)
org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
scala.Option.getOrElse(Option.scala:120)
org.apache.spark.rdd.RDD.partitions(R

In [8]:
val classifier = new LogisticRegression().setMaxIter(10).setTol(1E-6).setFitIntercept(true)
inputData.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = false)



In [9]:
val ovr = new OneVsRest()
ovr.setClassifier(classifier)

oneVsRest_1f546bd6df4f

In [10]:
val ovrModel = ovr.fit(train)

Name: Compile Error
Message: <console>:33: error: not found: value train
         val ovrModel = ovr.fit(train)
                                ^
StackTrace: 

In [11]:
val predictions = ovrModel.transform(test)
val predictionsAndLabels = predictions.select("prediction", "label").map(row => (row.getDouble(0), row.getDouble(1)))
predictions.show(100)
predictions.take(1)(0).get(1)

Name: Compile Error
Message: <console>:31: error: not found: value ovrModel
         val predictions = ovrModel.transform(test)
                           ^
StackTrace: 

In [12]:
val metrics = new MulticlassMetrics(predictionsAndLabels)
val confusionMatrix = metrics.confusionMatrix

Name: Compile Error
Message: <console>:31: error: not found: value predictionsAndLabels
         val metrics = new MulticlassMetrics(predictionsAndLabels)
                                             ^
StackTrace: 

In [13]:
println(s" Confusion Matrix\n ${confusionMatrix.toString}\n")

Name: Compile Error
Message: <console>:32: error: not found: value confusionMatrix
              println(s" Confusion Matrix\n ${confusionMatrix.toString}\n")
                                              ^
StackTrace: 

# Corpusreader

In [7]:
import java.io.IOError
import org.apache.log4j.LogManager
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{Accumulator, SparkContext}

import se.lth.cs.docforia.Document
import se.lth.cs.docforia.memstore.MemoryDocumentIO

object CorpusReader {

  /** Returns an RDD of [[se.lth.cs.docforia.Document]]
   *
   *  @param file - the path to the corpus
   *  @param sampleSize - sample this fraction of the corpus (default 1)
   */
  def readCorpus(
    file: String,
    sampleSize: Double = 1.0)
    (implicit sqlContext: SQLContext, sc: SparkContext): RDD[Document] = {

    val log = LogManager.getLogger(CorpusReader.getClass)
    var df: DataFrame = sqlContext.read.parquet(file)
    df = df.where(df("type").equalTo("ARTICLE"))

    val ioErrors: Accumulator[Int] = sc.accumulator(0, "IO_ERRORS")

    // we might need to filter for only articles here but that wouldn't be a generelized solution.
    val docs = (if(sampleSize == 1.0) df else df.sample(false, sampleSize)).flatMap{row =>
      try {
        val doc: Document = MemoryDocumentIO.getInstance().fromBytes(row.getAs(5): Array[Byte])
        List(doc)
      } catch {
        case e:IOError =>
          ioErrors.add(1)
          List()
      }
    }
    println(s"$ioErrors IO Errors encountered")
    docs
  }
}


# RelationsReader

In [10]:
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SQLContext

case class EntityPair(source: String, dest: String)
case class Relation(name: String, id: String, classIdx: Int, entities: Seq[EntityPair] = Seq())

/** Reads relations data file into an RDD of [[Relation]]
 */
object RelationsReader {
  def readRelations(file: String)(implicit sqlContext: SQLContext): RDD[Relation] = {
    import sqlContext.implicits._
    sqlContext.read.parquet(file).as[Relation].rdd
  }
}

# FeatureExtractor

In [20]:
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SQLContext
import se.lth.cs.docforia.Document
import se.lth.cs.docforia.graph.disambig.NamedEntityDisambiguation
import se.lth.cs.docforia.graph.text.{NamedEntity, Token}
import se.lth.cs.docforia.query.QueryCollectors


import scala.collection.JavaConverters._
import scala.collection.mutable



case class TrainingDataPoint(
  relationId: String,
  relationName: String,
  relationClass: Long,
  wordFeatures: Seq[String],
  posFeatures: Seq[String],
  ent1PosTags: Seq[String],
  ent2PosTags: Seq[String],
  ent1Type: String,
  ent2Type: String)

case class TestDataPoint(
  sentence: Document,
  qidSource: String,
  qidDest: String,
  wordFeatures: Seq[String],
  posFeatures: Seq[String],
  ent1PosFeatures: Seq[String],
  ent2PosFeatures: Seq[String],
  ent1Type: String,
  ent2Type: String)

case class FeatureArray(
  sentence: Document,
  subj: String,
  obj: String,
  wordFeatures: Seq[String],
  posFeatures: Seq[String],
  ent1PosFeatures: Seq[String],
  ent2PosFeatures: Seq[String],
  ent1Type: String,
  ent2Type: String)

case class TrainingSentence(
  relationId: String,
  relationName: String,
  relationClass: Int,
  sentenceDoc: Document,
  entityPair: Seq[EntityPair])



/** Extracts features for training/prediction
 */
object FeatureExtractor {

  val EMPTY_TOKEN = "<empty>"
  val NBR_WORDS_BEFORE = 3
  val NBR_WORDS_AFTER = 3
  val MIN_FEATURE_LENGTH = 2

  /** Returns an RDD of [[TrainingDataPoint]]
    *
    * Use this to collect training data for [[RelationModel]]
    *
    * @param trainingSentences  - an RDD of [[TrainingSentence]]
    */
  def trainingData(trainingSentences: RDD[TrainingSentence])(implicit sqlContext: SQLContext): RDD[TrainingDataPoint] = {

    val trainingPoints = trainingSentences.flatMap(t => {
      val neds = new mutable.HashSet() ++ t.entityPair.flatMap(p => Seq(p.source, p.dest))
      val featureArrays = featureArray(t.sentenceDoc).flatMap(f => {
        if (f.wordFeatures.length >= MIN_FEATURE_LENGTH) {
          if (neds.contains(f.subj) && neds.contains(f.obj)) {
            Seq(TrainingDataPoint(
              t.relationId,
              t.relationName,
              t.relationClass,
              f.wordFeatures,
              f.posFeatures,
              f.ent1PosFeatures,
              f.ent2PosFeatures,
              f.ent1Type,
              f.ent2Type))
          } else {
            Seq(TrainingDataPoint("neg", "neg", 0, f.wordFeatures, f.posFeatures, f.ent1PosFeatures, f.ent2PosFeatures,
              f.ent1Type, f.ent2Type))
          }
        } else {
          Seq()
        }
      })
      featureArrays
    })

    trainingPoints.repartition(12)
  }

  /** Returns an RDD of [[TestDataPoint]]
    *
    *   Use this to collect test data for [[RelationModel]]
    *
    *   @param sentences  - a Seq of Docforia Documents
    */
  def testData(sentences: Seq[Document])(implicit sqlContext: SQLContext): Seq[TestDataPoint] = {

    val testPoints = sentences.flatMap(sentence => {
      featureArray(sentence).map(f => {
        TestDataPoint(sentence, f.subj, f.obj, f.wordFeatures, f.posFeatures, f.ent1PosFeatures, f.ent2PosFeatures,
          f.ent1Type, f.ent2Type)
      })
    })

    testPoints
  }

  private def featureArray(sentence: Document): Seq[FeatureArray] = {

    val NED = NamedEntityDisambiguation.`var`()
    val NE = NamedEntity.`var`()
    val T = Token.`var`()

    sentence.nodes(classOf[Token])
      .asScala
      .toSeq
      .zipWithIndex
      .foreach(t => t._1.putTag("idx", t._2))

    val features = sentence.select(NED, T, NE)
      .where(T)
      .coveredBy(NED)
      .where(T)
      .coveredBy(NE)
      .stream()
      .collect(QueryCollectors.groupBy(sentence, NED, NE).values(T).collector())
      .asScala
      .toSet
      .subsets(2)
      .map(set => {
        /*
        Find the positions of the entities
         */
        val grp1 :: grp2 :: _ = set.toList

        val start1 = grp1.value(0, T).getTag("idx"): Int
        val end1 = grp1.value(grp1.size() - 1, T).getTag("idx"): Int

        val start2 = grp2.value(0, T).getTag("idx"): Int
        val end2 = grp2.value(grp2.size() - 1, T).getTag("idx"): Int

        /* Windows of words and POS */
        val words = tokenWindow(sentence, start1, end1, start2, end2, t => t.text)
        val pos = tokenWindow(sentence, start1, end1, start2, end2, t => t.getPartOfSpeech)

        /* Entity POS */
        val ent1TokensPos = grp1.nodes[Token](T).asScala.toSeq.map(t => t.getPartOfSpeech).toArray
        val ent2TokensPos = grp2.nodes[Token](T).asScala.toSeq.map(t => t.getPartOfSpeech).toArray

        /* Entity Mention Type */
        val ent1Type = if (grp1.key(NE).hasLabel) grp1.key(NE).getLabel else "<missing label>"
        val ent2Type = if (grp2.key(NE).hasLabel) grp2.key(NE).getLabel else "<missing label>"

        FeatureArray(
          sentence,
          grp1.key(NED).getIdentifier.split(":").last,
          grp2.key(NED).getIdentifier.split(":").last,
          words,
          pos,
          ent1TokensPos,
          ent2TokensPos,
          ent1Type,
          ent2Type)
      }).toSeq

    features

  }

  /** Extract string features from a Token window around two entities.
    */
  private def tokenWindow(sentence: Document, start1: Int, end1: Int, start2: Int, end2: Int, f: Token => String): Seq[String] = {
    /*
      Extract words before and after entity 1
     */
    val wordsBefore1 = sentence.nodes(classOf[Token]).asScala.toSeq.slice(start1 - NBR_WORDS_BEFORE, start1)
    val wordsAfter1 = sentence.nodes(classOf[Token]).asScala.toSeq.slice(end1 + NBR_WORDS_AFTER, end1 + NBR_WORDS_AFTER + 1)

    /*
      Extract words before and after entity 2
     */
    val wordsBefore2 = sentence.nodes(classOf[Token]).asScala.toSeq.slice(start2 - NBR_WORDS_BEFORE, start2)
    val wordsAfter2 = sentence.nodes(classOf[Token]).asScala.toSeq.slice(end2 + NBR_WORDS_AFTER, end2 + NBR_WORDS_AFTER + 1)

    /*
      Create string feature vector for the pair
     */
    val features = Seq(
      Seq.fill(NBR_WORDS_BEFORE - wordsBefore1.length)(EMPTY_TOKEN) ++ wordsBefore1.map(f),
      wordsAfter1.map(f) ++ Seq.fill(NBR_WORDS_AFTER - wordsAfter1.length)(EMPTY_TOKEN),
      Seq.fill(NBR_WORDS_BEFORE - wordsBefore2.length)(EMPTY_TOKEN) ++ wordsBefore2.map(f),
      wordsAfter2.map(f) ++ Seq.fill(NBR_WORDS_AFTER - wordsAfter2.length)(EMPTY_TOKEN)
    ).flatten
    features
  }

  /** Saves the training data to the path
   */
  def save(data: RDD[TrainingDataPoint], path: String)(implicit sqlContext: SQLContext): Unit = {
    import sqlContext.implicits._
    data.toDF().write.json(path)
  }

  /** Loads the data from path
   */
  def load(path: String)(implicit sqlContext: SQLContext): RDD[TrainingDataPoint]  = {
    import sqlContext.implicits._
    sqlContext.read.json(path).as[TrainingDataPoint].rdd
  }

}



# TrainingDataExtractor

In [28]:
import org.apache.log4j.LogManager
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.SQLContext
import se.lth.cs.docforia.Document
import se.lth.cs.docforia.graph.disambig.NamedEntityDisambiguation
import se.lth.cs.docforia.graph.text.Sentence
import se.lth.cs.docforia.memstore.MemoryDocumentIO
import se.lth.cs.docforia.query.QueryCollectors


import scala.collection.JavaConverters._
import scala.collection.mutable
import scala.collection.mutable.ListBuffer


case class TrainingSentence(
  relationId: String,
  relationName: String,
  relationClass: Int,
  sentenceDoc: Document,
  entityPair: Seq[EntityPair])

private case class SerializedTrainingSentence(
  relationId: String,
  relationName: String,
  relationClass: Int,
  sentenceDoc: Array[Byte],
  entityPair: Seq[EntityPair])


/** Provides training data extraction
 */
object TrainingDataExtractor {

  val SENTENCE_MAX_LENGTH = 220
  val SENTENCE_MIN_LENGTH = 5

  def printInfo(docs: RDD[Document], relations: RDD[Relation], sentences: RDD[TrainingSentence]): Unit = {
    val log = LogManager.getLogger(TrainingDataExtractor.getClass)
    val docsCount = docs.count()
    val relationsCount = relations.count()
    val entitiesCount = relations.map(r => (r.id -> r.entities.length)).collect().toMap
    //val dist = sentences.map(t => (t.relationId, 1)).reduceByKey(_ + _)
    //  .map(t => s"${t._1}\t-> ${entitiesCount.getOrElse(t._1, 0)} entities\t-> ${t._2}").collect()

    log.info("Extracting Training Sentences")
    log.info(s"Documents: $docsCount")
    log.info(s"Relations: $relationsCount")
    //dist.map(log.info)
  }

  /**
   * Extracts RDD of [[TrainingSentence]]
   */
  def extract(docs: RDD[Document], relations: RDD[Relation])(implicit sparkContext: SparkContext): RDD[TrainingSentence] = {

    val relMapping = relations.map(r => {
      // val entPairMap = new Object2ObjectOpenHashMap[String, mutable.Set[String]]()
      val entPairMap = scala.collection.mutable.Map[String, mutable.Set[String]]()
        r.entities.foreach(ep =>{
          val dests = entPairMap.getOrElse(ep.source, new mutable.HashSet[String]())
          dests += ep.dest
          entPairMap.put(ep.source, dests)
        })
      (r, entPairMap)
    })

    val broadcastRM = sparkContext.broadcast(relMapping.collect())

    val data:RDD[TrainingSentence] = docs.flatMap(doc => {

        val S = Sentence.`var`()
        val NED = NamedEntityDisambiguation.`var`()

        val trainingSentences:Seq[TrainingSentence] = doc.select(S, NED)
          .where(NED)
          .coveredBy(S)
          .stream()
          .collect(QueryCollectors.groupBy(doc, S).values(NED).collector()).asScala
          .filter(pg => SENTENCE_MIN_LENGTH <= pg.key(S).length() && pg.key(S).length() <= SENTENCE_MAX_LENGTH)
          .flatMap(pg => {

            val neds = pg.list(NED).asScala.map(_.getIdentifier.split(":").last).toSet.subsets(2).map(_.toSeq).toSeq
            lazy val sDoc = doc.subDocument(pg.key(S).getStart, pg.key(S).getEnd)

            val trainingData = broadcastRM.value.flatMap{
              case (relation, mapping) => {
                val pairs = neds.flatMap(pair => {
                  val foundPairs = ListBuffer[EntityPair]()
                  if (mapping.getOrElse(pair(0), mutable.Set.empty).contains(pair(1))){
                    foundPairs += EntityPair(pair(0), pair(1))
                  } else if (mapping.getOrElse(pair(1), mutable.Set.empty).contains(pair(0))){
                    foundPairs += EntityPair(pair(1), pair(0))
                  }
                  foundPairs
                })

                if (pairs.length > 0)
                  Seq(TrainingSentence(relation.id, relation.name, relation.classIdx, sDoc, pairs.toList))
                else
                  Seq()
              }
            }

            trainingData
          })

        trainingSentences
      })

    data.repartition(12)
  }

}



In [30]:
val docs = CorpusReader.readCorpus("/Users/axel/utveckling/prometheus/data/wikipedia-corpus-herd/sv")(sqlContext, sc)
val relations = RelationsReader.readRelations("/Users/axel/utveckling/prometheus/data/0.4.0/entities")(sqlContext)
val sentences = TrainingDataExtractor.extract(docs, relations)(sc)

0 IO Errors encountered


Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace: org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:304)
org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:294)
org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:122)
org.apache.spark.SparkContext.clean(SparkContext.scala:2055)
org.apache.spark.rdd.RDD$$anonfun$flatMap$1.apply(RDD.scala:333)
org.apache.spark.rdd.RDD$$anonfun$flatMap$1.apply(RDD.scala:332)
org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:150)
org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:111)
org.apache.spark.rdd.RDD.withScope(RDD.scala:316)
org.apache.spark.rdd.RDD.flatMap(RDD.scala:332)
$line276.$read$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$$iwC$TrainingDataExtr

In [2]:
case class TrainingSentence(
  relationId: String,
  relationName: String,
  relationClass: Int,
  sentenceDoc: Document,
  entityPair: Seq[EntityPair])

private case class SerializedTrainingSentence(
  relationId: String,
  relationName: String,
  relationClass: Int,
  sentenceDoc: Array[Byte],
  entityPair: Seq[EntityPair])

val trainingSentences = sqlContext.read.parquet("/Users/axel/utveckling/prometheus/data/0.1.0-32-g4d0b685/relation_model/en/training_sentences")
rawData.map(st => {
  TrainingSentence(st.relationId, st.relationName, st.relationClass,
                   MemoryDocumentIO.getInstance().fromBytes(st.sentenceDoc), st.entityPair)
})


trainingSentences.printSchema()
trainingSentences.take(1).

Name: Compile Error
Message: <console>:19: error: not found: type Document
         sentenceDoc: Document,
                      ^
<console>:20: error: not found: type EntityPair
         entityPair: Seq[EntityPair])
                         ^
StackTrace: 